1. Imputación de la Moda: Reemplazo los valores Missing con la moda.

2. Codificación Ordinal: Las columnas jerárquicas se codifican con valores 1, 2, 3, 4 

3. One-Hot Encoding:  Las variables categóricas restantes se transforman en columnas binarias.

4. Realizo la normalización de las variables despuse del one-hot-encoding para asegurar que las variables codificadas aporten igual importancia al cálculo de distancias, de esta manera el clustering será justo y equilibrado porque ninguna variable dominará el proceso debido a su escala o rango. 

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import linear_model 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.preprocessing import scale, StandardScaler, OneHotEncoder
import sklearn
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

In [23]:
df = pd.read_csv("data_3.csv")
df.head(5)

,esg_score21,esg_class21,environ_score21,environ_class21,social_score21,social_class21,govern_score21,govern_class21,turnover21,country,...,esg_exe_remuneration,esg_respect_humanr,esg_human_rights,esg_respect_freedom,esg_non_discrimination,esg_physical_risk,esg_energy_transition,esg_carbon_emission1,esg_carbon_emission2,esg_carbon_emission1_2
0,-0.569247,Weak performance,-1.721908,Weak performance,-1.041523,Weak performance,0.656902,Limited performance,1.061682,USA,...,0.614529,-1.304142,-0.000852,-0.813810,0.151979,-0.761158,-1.539686,0.016030,0.054868,-0.777691
1,0.534502,Weak performance,0.446051,Weak performance,-0.150737,Weak performance,1.204951,Limited performance,1.361008,USA,...,2.139914,-0.113515,0.403974,-0.601699,0.686580,2.077778,0.207338,-0.530961,1.017824,0.489308
2,1.454293,Limited performance,0.686935,Weak performance,1.096364,Weak performance,2.301049,Limited performance,-0.059054,USA,...,2.444991,1.870863,1.213625,-0.389588,1.933982,2.077778,-0.195822,-1.018312,-0.001435,-0.585596
3,0.166586,Weak performance,-0.637929,Weak performance,-0.328894,Weak performance,0.930926,Limited performance,-0.582949,USA,...,0.716222,0.547944,0.269032,-0.601699,0.864780,-0.534043,-0.867754,0.067916,0.631736,0.148261
4,0.166586,Weak performance,0.084724,Weak performance,-0.685208,Weak performance,1.067939,Limited performance,-2.161638,USA,...,1.631453,0.018777,-0.000852,-1.662252,0.419279,-0.534043,-0.061435,-1.302281,-1.033073,-1.255900


One-Hot Encoding: Para las variables donde necesito preservar las diferencias entre categorias

Codificación Ordinal: Para variables donde hay relaciones de orden.


In [31]:
def procesar_dataframe(df, output_file):
    """
    Función para procesar un DataFrame:
    1. Eliminar las columnas 'activity' y 'type_event'.
    2. Codificación ordinal para variables jerárquicas.
    3. Imputación de valores missing con la moda.
    4. One-Hot Encoding para variables categóricas.
    5. Normalización de todas las variables numéricas.
    6. Guarda el resultado en un archivo CSV.
    """
    # Eliminar las columnas 'activity' y 'type_event'
    df = df.drop(columns=['activity', 'type_event'], errors='ignore')
    print("Columnas 'activity' y 'type_event' eliminadas.")

    # 1. Codificación ordinal para variables con jerarquía
    esg_mapping = {'Weak performance': 1, 
                   'Limited performance': 2, 
                   'Robust performance': 3, 
                   'Advanced performance': 4}
    for col in ['esg_class21', 'environ_class21', 'social_class21', 'govern_class21']:
        moda = df[col][df[col] != 'Missing'].mode()[0]
        df[col] = df[col].replace('Missing', np.nan).fillna(moda)
        df[col] = df[col].map(esg_mapping)

    # 2. One-Hot Encoding para variables categóricas con imputación de valores missing
    variables_categoricas = ['country', 'region', 'sectors', 'audit_status21']
    for col in variables_categoricas:
        moda = df[col].mode()[0]
        df[col] = df[col].fillna(moda)

    df_encoded = pd.get_dummies(df, columns=variables_categoricas, drop_first=True)

    # 3. Normalización de todas las variables numéricas
    scaler = StandardScaler()
    data_4 = pd.DataFrame(scaler.fit_transform(df_encoded), columns=df_encoded.columns)

    # 4. Guardar el DataFrame en un archivo CSV
    data_4.to_csv(output_file, index=False)
    print(f"Archivo guardado como: {output_file}")
    print("Shape del DataFrame normalizado:", data_4.shape)
    print(data_4.head())

    return df_normalizado

# Uso de la función
# Asegúrate de tener el DataFrame df cargado antes de llamar a la función
output_file = 'data_4.csv'
df_normalizado = procesar_dataframe(df, output_file)

Columnas 'activity' y 'type_event' eliminadas.
Archivo guardado como: data_4.csv
Shape del DataFrame normalizado: (22581, 139)
   esg_score21  esg_class21  environ_score21  environ_class21  social_score21  \
0    -0.569247    -0.345854        -1.721908        -0.231856       -1.041523   
1     0.534502    -0.345854         0.446051        -0.231856       -0.150737   
2     1.454293     2.325055         0.686935        -0.231856        1.096364   
3     0.166586    -0.345854        -0.637929        -0.231856       -0.328894   
4     0.166586    -0.345854         0.084724        -0.231856       -0.685208   

   social_class21  govern_score21  govern_class21  turnover21  firm_age  ...  \
0        -0.34062        0.656902        0.766337    1.061682 -1.131157  ...   
1        -0.34062        1.204951        0.766337    1.361008  0.050243  ...   
2        -0.34062        2.301049        0.766337   -0.059054 -0.377435  ...   
3        -0.34062        0.930926        0.766337   -0.582949 -2.0

Al realizar el one-hot-encoding me quedo con 190 variables 